# Spine Model

Differential equation for the Calcium concentration at the post synaptic spine.

In [1]:
# Model and simulation parameters

freq = 5 #Hz
const τ₁ = 50.0
const τ₂ = 5.0
const τ_b = 20.0
const τ_k = 56.0
const τ_Ca = 50.0
const vₑ = 1.0
const v_b = 60.0
const v_k = 2.0
const vᵣ = - 65.0
const Vᵣₑᵥ = 130.0
const α = 5.0
const K_d = 0.32
const P₀ = 10.0
const ḡ = 0.00103
const t_est = collect(1:1000/freq:1000)
const τ_f = 200.0
const τ_s = 50.0
const I_f = 0.5
const I_s = 0.5

0.5

Defining the differential equation goberning the calcium concentration:

$$\frac{d[Ca^{2+}]}{dt} = - I_{NMDA}(t) - \frac{1}{\tau_{Ca}} [Ca^{2+}]$$


In [2]:
dCa(Ca, Δt, t) = - Inmda(Ca, t, t_est, Δt) - (1/τ_Ca) * Ca

dCa (generic function with 1 method)

$$I_{NMDA}(t,\{t_{pre}\},\{t_{post}\}) = \bar{g}f(t,t_{pre})\mathcal{H}(V)$$

In [3]:
Inmda(Ca, t, t_pre, Δt) = ḡ * f(t, t_pre) * ℋ(V(Ca, t, t_pre, Δt))

Inmda (generic function with 1 method)

$$f(t,t_{pre})=P_0[I_fΘ(t−t_{pre})e^{−(t−t_{pre})/τf} +I_sΘ(t−t_{pre})e^{−(t−t_{pre})/τs}]$$

In [4]:
f(t, t_pre) = P₀ * (I_f * Θ(t - t_pre) * exp(-(t - t_pre)/τ_f) + I_s * Θ(t - t_pre) * exp(-(t - t_pre)/τ_s))
f(t, t_pre::Vector) = sum(f.(t, t_pre))

f (generic function with 2 methods)

$$\mathcal{H}(V) = \frac{V(t) - V_{rev}}{1 + \frac{e^{-0.062V}}{3.59}}$$

In [5]:
ℋ(V) = (V - Vᵣₑᵥ) / (1 + (exp(-0.062 * V)/3.57))

ℋ (generic function with 1 method)

$$V (t) = v_R + BPAP(t) + EPSP_{AMPA}(t) + EPSP_{SK}(t)$$

In [6]:
V(Ca, t, t_pre, Δt) = vᵣ + BPAP(t, t_pre, Δt) + EPSP_AMPA(t, t_pre) + EPSP_SK(Ca, t, t_pre, Δt)

V(Ca, t, t_pre::Vector, Δt) = vᵣ + sum(BPAP.(t, t_pre, Δt) + EPSP_AMPA.(t, t_pre) + EPSP_SK.(Ca, t, t_pre, Δt))

V (generic function with 2 methods)

$$BPAP(t,t_{post}) = v_B Θ(t − t_{post}) e^{\frac{−(t−tpost)}{τ_B}}$$

In [7]:
# Defining a Heaviside function
Θ(x) = x < zero(x) ? zero(x) : one(x)

Θ (generic function with 1 method)

In [8]:
BPAP(t, t_pre, Δt) = v_b * Θ(t - (t_pre + Δt)) * exp(-(t - (t_pre + Δt))/τ_b)

BPAP (generic function with 1 method)

$$ESPS_{AMPA}(t) = v_E Θ(t−t_{pre})􏰂 \bigg[e^{\frac{−(t−t_{pre})}{τ_1}} +e^{\frac{−(t−t_{pre})}{τ_2}}\bigg]􏰃$$

In [9]:
EPSP_AMPA(t, t_pre) = vₑ * Θ(t - t_pre) * (exp(-(t - t_pre)/τ₁) + exp(-(t - t_pre)/τ₂))

EPSP_AMPA (generic function with 1 method)

$$EPSP_{SK} = v_{SK} \bigg[ \frac{[Ca^{2+}]^\alpha}{[Ca^{2+}]^\alpha K^\alpha_d} \bigg]$$

$$v_{SK} = v_k Θ(t−t_{post})􏰂 e^{\frac{−(t−t_{post})}{τ_k}}$$

In [10]:
EPSP_SK(Ca, t, t_pre, Δt) = v_sk(t, t_pre, Δt) * (Ca^α / (Ca^α + K_d^α))

v_sk(t, t_pre, Δt) = v_k * Θ(t - (t_pre + Δt)) * exp(-(t - (t_pre + Δt))/τ_k) + 0.018

v_sk (generic function with 1 method)

Defining the ODE system

In [11]:
using OrdinaryDiffEq

In [12]:
Ca₀ = 0.0
t = 0.0:1:1000.0
Δt = 10.0
prob = ODEProblem(dCa, Ca₀, (0.0,1000.0), Δt, saveat = t)
sol1 = solve(prob, Tsit5())

Δt = -20.0
prob = remake(prob, p = Δt)
sol2 = solve(prob, Tsit5());

In [13]:
using Plots

In [ ]:
plot(sol1 , label = "Δt = 10 ms", title = "5 Hz")
plot!(sol2 , label = "Δt = -20 ms")

In [ ]:
using Interact

In [ ]:
@Interact.manipulate for Δt ∈ -100:1.0:100
    Ca₀ = 0.0
    t = 0.0:1:1000.0
    prob = ODEProblem(dCa, Ca₀, (0.0,1000.0), Δt, saveat = t)
    sol = solve(prob, Tsit5())
    plot(sol, label = "Δt = $Δt ms", title = "5 Hz")
    ylims!(0,15.0)
end